In [1]:
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import uuid
import cv2
import numpy as np
from random import randint
import random
from PIL import Image
import json
import copy

from tqdm.notebook import tqdm
from time import sleep

In [4]:
!ls

bee_flowers_data                   json_to_amazon_ground_truth.ipynb
example_video.mp4                  mosaic_data
final_data+synth+mosaic            mosaic_data_augmentation.ipynb
final_data+synth+mosaic_rebalanced other_noinsect
flickr_api_key.json                raw
flickr_dataset_collection.ipynb    synthetic_data
flowers_data                       video_data


In [5]:
with open("./video_data/bees_demo1_AWS_split/via_export_json_AWS.json") as json_file: #read annotation data
    annotation_data = json.load(json_file)
json_file.close()

In [8]:
#annotation_data

In [11]:
bee_entry_names = [name for name in annotation_data]

In [25]:
annotation_data[bee_entry_names[1]]['filename']

'bees_demo1_frame1.jpg'

In [50]:
def get_bb_anno(height, width, x, y, ID):
    bb_anno = {"height":height,"width":width,"top":y + height,"left":x,"class-id":"0",
           "label-category-attributes":{},
           "object-id":"%04dfc20-e807-11eb-8483-47df3d5f0ebb" % ID,
           "object-name":f"bee:{ID}"}
    return bb_anno

In [51]:
def get_regions(regions):
    bee_ID = 1
    annots = []
    for region in regions:
        attr = region['shape_attributes']
        x = attr["x"]
        y = attr["y"]
        width = attr["width"]
        height = attr["height"]
        annots.append(get_bb_anno(height, width, x, y, bee_ID))
        bee_ID += 1
    return annots

In [52]:
frame_no = 0
frame_annots = []
for name in bee_entry_names:
    filename = annotation_data[name]['filename']
    regions = annotation_data[name]['regions']
    frame_anno = {"annotations": get_regions(regions),
    "frame-no":f"{frame_no}",
    "frame":"bee_video_frame%03d.jpg" % frame_no,
    "frame-attributes":{}}
    frame_annots.append(frame_anno)
    frame_no += 1

In [55]:
frame_annots[0]

{'annotations': [{'height': 46,
   'width': 49,
   'top': 539,
   'left': 387,
   'class-id': '0',
   'label-category-attributes': {},
   'object-id': '0001fc20-e807-11eb-8483-47df3d5f0ebb',
   'object-name': 'bee:1'},
  {'height': 50,
   'width': 67,
   'top': 204,
   'left': 303,
   'class-id': '0',
   'label-category-attributes': {},
   'object-id': '0002fc20-e807-11eb-8483-47df3d5f0ebb',
   'object-name': 'bee:2'},
  {'height': 49,
   'width': 41,
   'top': 376,
   'left': 677,
   'class-id': '0',
   'label-category-attributes': {},
   'object-id': '0003fc20-e807-11eb-8483-47df3d5f0ebb',
   'object-name': 'bee:3'},
  {'height': 35,
   'width': 38,
   'top': 372,
   'left': 589,
   'class-id': '0',
   'label-category-attributes': {},
   'object-id': '0004fc20-e807-11eb-8483-47df3d5f0ebb',
   'object-name': 'bee:4'},
  {'height': 32,
   'width': 77,
   'top': 33,
   'left': 120,
   'class-id': '0',
   'label-category-attributes': {},
   'object-id': '0005fc20-e807-11eb-8483-47df3d5f0

In [58]:
final_frame_annot_dict = {"tracking-annotations":frame_annots}

In [62]:
with open("./video_data/bees_demo1_AWS_split/SeqLabel.json", "w") as file:
    json.dump(final_frame_annot_dict, file)

In [ ]:
{"tracking-annotations":[
    {"annotations":[{"height":34,"width":45,"top":102,"left":512,"class-id":"0","label-category-attributes":{"ID":"0"},"object-id":"245d4700-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:1"},{"height":41,"width":32,"top":333,"left":591,"class-id":"0","label-category-attributes":{},"object-id":"2dddfc70-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:2"},{"height":50,"width":39,"top":331,"left":677,"class-id":"0","label-category-attributes":{},"object-id":"34752680-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:3"},{"height":50,"width":66,"top":67,"left":982,"class-id":"0","label-category-attributes":{},"object-id":"427dfc20-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:4"}],
    "frame-no":"1",
    "frame":"bees_demo1_frame1.jpg",
    "frame-attributes":{}},
    {"annotations":[{"height":32,"width":42,"top":95,"left":495,"class-id":"0","label-category-attributes":{"ID":"0"},"object-id":"245d4700-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:1"},{"height":41,"width":32,"top":331,"left":590,"class-id":"0","label-category-attributes":{},"object-id":"2dddfc70-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:2"},{"height":49,"width":38,"top":331,"left":676,"class-id":"0","label-category-attributes":{},"object-id":"34752680-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:3"}],"frame-no":"2","frame":"bees_demo1_frame10.jpg","frame-attributes":{}},
    {"annotations":[{"height":34,"width":45,"top":102,"left":512,"class-id":"0","label-category-attributes":{"ID":"0"},"object-id":"245d4700-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:1"},{"height":41,"width":32,"top":333,"left":591,"class-id":"0","label-category-attributes":{},"object-id":"2dddfc70-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:2"},{"height":50,"width":39,"top":331,"left":677,"class-id":"0","label-category-attributes":{},"object-id":"34752680-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:3"},{"height":50,"width":66,"top":67,"left":982,"class-id":"0","label-category-attributes":{},"object-id":"427dfc20-e807-11eb-8483-47df3d5f0ebb","object-name":"bee:4"}],"frame-no":"3","frame":"bees_demo1_frame100.jpg","frame-attributes":{}}
]}
